In [66]:
import boto3
import pandas as pd
from io import StringIO
from datetime import datetime, timedelta

In [67]:
arg_date = '2022-03-15'
src_format = '%Y-%m-%d'
source_bucket = 'xetra-1234'
trg_bucket = 'josh-123456'
columns=['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume']
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

In [68]:
arg_date_dt = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)

In [69]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(source_bucket)
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0],src_format).date() >= arg_date_dt]

In [70]:
def csv_to_df(filename):
    csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    return df
df_all = pd.concat([csv_to_df(obj.key) for obj in objects[:15]], ignore_index=True)  
#df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index=True)

C:\Users\JOSUE\AppData\Local\Temp\ipykernel_16932\2635660564.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([csv_to_df(obj.key) for obj in objects[:15]], ignore_index=True)


In [71]:
df_all

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,AT0000A0E9W5,SANT,S+T AG O.N.,Common stock,EUR,2504159,2022-03-14,08:00,12.4000,12.4900,12.4000,12.4900,2717,3
1,DE000A0DJ6J9,S92,SMA SOLAR TECHNOL.AG,Common stock,EUR,2504287,2022-03-14,08:00,40.8000,41.1200,40.8000,41.0800,3639,11
2,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2022-03-14,08:00,17.8200,17.9000,17.6000,17.9000,43107,52
3,DE000A0HN5C6,DWNI,DEUTSCHE WOHNEN SE INH,Common stock,EUR,2504314,2022-03-14,08:00,35.4000,35.4000,35.3800,35.4000,3563,20
4,DE000A0JL9W6,VBK,VERBIO VER.BIOENERGIE ON,Common stock,EUR,2504343,2022-03-14,08:00,64.0000,65.2500,64.0000,64.6500,1495,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98238,LU2333563281,TNIE,TONIES SE EO 1,Common stock,EUR,6356244,2022-03-14,14:59,6.6000,6.6000,6.6000,6.6000,827,1
98239,DE000A3CNK42,YOU,ABOUT YOU HOLDING SE,Common stock,EUR,6501749,2022-03-14,14:59,11.9700,11.9700,11.9500,11.9500,58,3
98240,DE000DTR0CK8,DTG,DAIMLER TRUCK HLDG JGE NA,Common stock,EUR,7126155,2022-03-14,14:59,24.6250,24.6250,24.6050,24.6100,1202,8
98241,DE000A3GVKX6,PLKA,ETC ISSUANCE O.END ETN,ETN,EUR,7135039,2022-03-14,14:59,4.0022,4.0022,4.0022,4.0022,1200,1


In [72]:
df_all.shape

(98243, 14)

In [73]:
df_all = df_all.loc[:, columns]

In [74]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

In [75]:
df_all[df_all['ISIN']== 'DE000A0DJ6J9']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
1,DE000A0DJ6J9,2022-03-14,08:00,40.80,41.12,40.80,41.08,3639,40.8
103,DE000A0DJ6J9,2022-03-14,08:01,40.94,40.94,40.56,40.70,685,40.8
189,DE000A0DJ6J9,2022-03-14,08:02,40.86,41.20,40.86,41.20,1181,40.8
406,DE000A0DJ6J9,2022-03-14,08:03,41.26,41.26,40.80,40.90,489,40.8
579,DE000A0DJ6J9,2022-03-14,08:04,40.72,41.10,40.72,41.10,438,40.8
...,...,...,...,...,...,...,...,...,...
96929,DE000A0DJ6J9,2022-03-14,14:54,37.74,37.74,37.72,37.72,102,40.8
97134,DE000A0DJ6J9,2022-03-14,14:55,37.76,37.78,37.76,37.78,142,40.8
97366,DE000A0DJ6J9,2022-03-14,14:56,37.80,37.80,37.66,37.74,1079,40.8
97618,DE000A0DJ6J9,2022-03-14,14:57,37.70,37.76,37.70,37.76,77,40.8


In [76]:
df_all['Closing_Price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['EndPrice'].transform('last')

In [77]:
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,Closing_Price
0,AT0000A0E9W5,2022-03-14,08:00,12.4000,12.4900,12.4000,12.4900,2717,12.400,12.6800
1,DE000A0DJ6J9,2022-03-14,08:00,40.8000,41.1200,40.8000,41.0800,3639,40.800,37.8200
2,DE000A0D6554,2022-03-14,08:00,17.8200,17.9000,17.6000,17.9000,43107,17.820,16.9200
3,DE000A0HN5C6,2022-03-14,08:00,35.4000,35.4000,35.3800,35.4000,3563,35.400,35.0400
4,DE000A0JL9W6,2022-03-14,08:00,64.0000,65.2500,64.0000,64.6500,1495,64.000,63.6500
...,...,...,...,...,...,...,...,...,...,...
98238,LU2333563281,2022-03-14,14:59,6.6000,6.6000,6.6000,6.6000,827,6.650,6.6000
98239,DE000A3CNK42,2022-03-14,14:59,11.9700,11.9700,11.9500,11.9500,58,12.010,11.9500
98240,DE000DTR0CK8,2022-03-14,14:59,24.6250,24.6250,24.6050,24.6100,1202,24.740,24.6100
98241,DE000A3GVKX6,2022-03-14,14:59,4.0022,4.0022,4.0022,4.0022,1200,4.018,4.0022


In [78]:
df_all[df_all['ISIN']== 'DE000A0D9PT0']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,Closing_Price
191,DE000A0D9PT0,2022-03-14,08:02,201.3,203.2,201.3,202.7,5146,201.3,205.1
408,DE000A0D9PT0,2022-03-14,08:03,202.7,204.6,202.5,204.1,8223,201.3,205.1
586,DE000A0D9PT0,2022-03-14,08:04,204.1,204.7,203.4,204.0,3020,201.3,205.1
2834,DE000A0D9PT0,2022-03-14,08:05,204.3,204.6,203.3,203.7,2728,201.3,205.1
3216,DE000A0D9PT0,2022-03-14,08:06,203.7,203.9,203.0,203.7,1298,201.3,205.1
...,...,...,...,...,...,...,...,...,...,...
97135,DE000A0D9PT0,2022-03-14,14:55,204.9,205.1,204.9,205.1,120,201.3,205.1
97368,DE000A0D9PT0,2022-03-14,14:56,205.1,205.1,204.8,204.8,923,201.3,205.1
97620,DE000A0D9PT0,2022-03-14,14:57,204.7,204.7,204.6,204.6,85,201.3,205.1
97827,DE000A0D9PT0,2022-03-14,14:58,204.7,204.8,204.7,204.8,147,201.3,205.1


## Agregations

In [79]:
df_all = df_all.groupby(['ISIN','Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('Closing_Price', 'min'),minimun_price_eur=('MinPrice', 'min'), maximun_price_eur=('MaxPrice', 'min'), daily_traded_volumen=('TradedVolume','sum')) 

In [80]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [81]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] -df_all['prev_closing_price']) / df_all ['prev_closing_price'] * 100

In [82]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [83]:
df_all = df_all.round(decimals=2)

In [84]:
#df_all = df_all[df_all.Date >= arg_date]

In [85]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimun_price_eur,maximun_price_eur,daily_traded_volumen,change_prev_closing_%
0,AT000000STR1,2022-03-14,35.45,36.40,35.45,35.45,113,NaN
1,AT00000FACC2,2022-03-14,7.83,7.85,7.83,7.83,130,NaN
2,AT0000606306,2022-03-14,13.66,13.32,13.18,13.20,32139,NaN
3,AT0000609607,2022-03-14,11.36,11.40,11.36,11.36,200,NaN
4,AT0000644505,2022-03-14,91.40,92.20,91.40,91.40,2388,NaN
...,...,...,...,...,...,...,...,...
2964,XS2427363036,2022-03-14,22.38,21.78,21.78,21.78,0,NaN
2965,XS2427363895,2022-03-14,18.58,18.58,18.58,18.58,0,NaN
2966,XS2427474023,2022-03-14,24.01,24.25,24.01,24.01,10,NaN
2967,XS2434891219,2022-03-14,3.57,3.56,3.54,3.54,3936,NaN


## SAVE S3

In [86]:
from io import StringIO, BytesIO

In [87]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket(trg_bucket)
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='josh-123456', key='xetra_daily_report_20240326_203534.parquet')

## READING THEUPLOAD FILE

In [88]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report_20240326_202353.parquet
xetra_daily_report_20240326_203426.parquet
xetra_daily_report_20240326_203534.parquet


In [92]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20240326_203534.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [93]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimun_price_eur,maximun_price_eur,daily_traded_volumen,change_prev_closing_%
0,AT000000STR1,2022-03-14,35.45,36.40,35.45,35.45,113,NaN
1,AT00000FACC2,2022-03-14,7.83,7.85,7.83,7.83,130,NaN
2,AT0000606306,2022-03-14,13.66,13.32,13.18,13.20,32139,NaN
3,AT0000609607,2022-03-14,11.36,11.40,11.36,11.36,200,NaN
4,AT0000644505,2022-03-14,91.40,92.20,91.40,91.40,2388,NaN
...,...,...,...,...,...,...,...,...
2964,XS2427363036,2022-03-14,22.38,21.78,21.78,21.78,0,NaN
2965,XS2427363895,2022-03-14,18.58,18.58,18.58,18.58,0,NaN
2966,XS2427474023,2022-03-14,24.01,24.25,24.01,24.01,10,NaN
2967,XS2434891219,2022-03-14,3.57,3.56,3.54,3.54,3936,NaN
